<a href="https://colab.research.google.com/github/alineaprelini/Pyspark/blob/main/Spark_DML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pySpark

In [7]:
#Bibliotecas
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [8]:
#Iniciando Sessão
spark = SparkSession.builder \
    .appName("A") \
    .getOrCreate()

In [45]:
#Carregando base de dados
df1 = spark.read.csv('Marca.csv', header = True, sep = ',')
df2 = spark.read.csv('Produto.csv', header = True, sep = ',')
df3 = spark.read.csv('Clientes.csv', header = True, sep = ';')

In [46]:
df1.show(10)
df2.show(10)
df3.show(10)

+---+-------------+-----------+-------+----------+------------------+--------------------+
|_c0|    IdProduto|DescProduto|IdMarca|Quantidade|             Preço|           DescMarca|
+---+-------------+-----------+-------+----------+------------------+--------------------+
|  0|1245741840860|     Social|      5|       788|             15.41| Li, Smith and Scott|
|  1|9214404086305|      Point|      7|       924|47.720000000000006|Gonzales, Young a...|
|  2|6223853590489|   Evidence|      2|       383|              37.6|        Martin Group|
|  3|8251950562038|        Too|      1|       666|15.739999999999998|Lawrence, William...|
|  4|0868436639740|    Quickly|      9|       242|             79.35|Ayala, Mitchell a...|
|  5|7679841035569|       Door|     10|       697|             76.04|           Black LLC|
|  6|3610164719044|         Tv|      3|       933| 97.91000000000001|         Hardy-Allen|
|  7|3786459022783|     Summer|      6|        15|             45.28|     Sheppard-Duncan|

In [54]:
#Join Marca = produto
df4 = df2.join(df1, df2['IdMarca'] == df1['IdMarca'], how='inner').select(df1['IdMarca'], df1['DescMarca'], df2['IdProduto'], df2['DescProduto'], df2['Quantidade'], df2['Preço'])
df4.show()


+-------+--------------------+-------------+-----------+----------+------------------+
|IdMarca|           DescMarca|    IdProduto|DescProduto|Quantidade|             Preço|
+-------+--------------------+-------------+-----------+----------+------------------+
|      5|           Kim-Smith|1245741840860|     Social|       788|             15.41|
|      5|       Phelps-Butler|1245741840860|     Social|       788|             15.41|
|      5|Edwards, Williams...|1245741840860|     Social|       788|             15.41|
|      5|          Duran-Pace|1245741840860|     Social|       788|             15.41|
|      5|        Payne-Mathis|1245741840860|     Social|       788|             15.41|
|      5|Aguilar, Medina a...|1245741840860|     Social|       788|             15.41|
|      5|          Little LLC|1245741840860|     Social|       788|             15.41|
|      5|      Harris-Cabrera|1245741840860|     Social|       788|             15.41|
|      5|           Hines Inc|1245741840860

In [49]:
df4.printSchema

<bound method DataFrame.printSchema of DataFrame[_c0: string, IdProduto: string, DescProduto: string, IdMarca: string, Quantidade: string, Preço: string, _c0: string, IdProduto: string, DescProduto: string, IdMarca: string, Quantidade: string, Preço: string, DescMarca: string]>

In [59]:
df5 = df4.join(df3, df3['IdMarca'] == df4['IdMarca']).select(df3['IdCliente'],df3['Nome'], df3['Idade'], df3['País'], df3['Cidade'], df3['IdProduto'], df3['IdMarca'], df4['DescProduto'],df3['Quantidade'])
df5.show()

+---------+-----------------+-----+--------------------+------------------+---------+-------+-----------+----------+
|IdCliente|             Nome|Idade|                País|            Cidade|IdProduto|IdMarca|DescProduto|Quantidade|
+---------+-----------------+-----+--------------------+------------------+---------+-------+-----------+----------+
| 80934224|   Abigail Holmes|   35|              Poland|        Debbiefurt|        5|      5|     Social|         8|
| 60936668| Richard Anderson|   19|             Reunion|      West Matthew|        1|      5|     Social|         6|
| 21205014|     Antonio West|   51|               Gabon|       Loriborough|       10|      5|     Social|         5|
| 92005975|  Michael Blevins|   33|           Mauritius|West Angelicamouth|        6|      5|     Social|         2|
| 97694562|      Sharon Boyd|   25|              Cyprus|    Lake Jasonberg|        6|      5|     Social|         7|
| 41374434|Mr. Jonathan Kemp|   48|      American Samoa|        